## **Script do Pipeline de Carga das Cotações das Ações negociadas na Bolsa (Ibovespa/B3)**

Os detalhes das ações que você negocia na bolsa podem ser acessadas pela **Nota de Corretagem**, emitida pela corretora. 
A nota é um extrato das operações realizadas na Bolsa de Valores e contém detalhes como: 

* Espécie da operação
* Quantidade de títulos
* Preço
* Data do pregão
* Valor da negociação
* Corretagem cobrada
* Emolumentos devidos
* Mercado em que a operação foi realizada
* Se a operação foi de compra (C) ou venda (V)
* Tipo de mercado em que a operação foi realizada
* Especificações do título 



In [1]:
try: 
  import warnings    
  warnings.filterwarnings("ignore")
except ImportError:
   %pip install warnings
   import warnings   
   warnings.filterwarnings("ignore")

try:
  import pandas as pd
except ImportError:
    %pip install pandas
    import pandas

try:
  from tradingcomdados import alternative_data as ad
except ImportError:
    %pip install tradingcomdados
    from tradingcomdados import alternative_data as ad

try:
   from yahooquery import Ticker
except ImportError:
   %pip install yahooquery --upgrade --no-cache-dir
   from yahooquery import Ticker

try:
  import datetime as dt
except ImportError:
   %pip install datetime
   import datetime as dt

### **Mapeamento do Diretório de input das ações:**

**Layout:** 
* **TITULO** CORRETORA = Descrição do Titulo na corretora
* **CODIGO** = Código da ação para negociação na Bolsa
* **COTAS** = Quantidade de fração/Cotas compradas
* **VALOR** = Valor pago na Cota/Fração da Ação
* **DATA COMPRA** = Data da compra da Cota/Fração 

In [2]:
file = 'Inputs/MinhasAcoes.xlsx'
acoes_ibov = pd.read_excel(file, sheet_name='Planilha1') #minha tabela de ações

In [3]:
#Dataframe para armazenar o segmento das ações
dfAcoes = pd.DataFrame(columns = ['SETORECONOMICO','SUBSETOR','SEGMENTO','EMPRESA','TITULO','ACAO','QTDECOTAS','VALOR','DATACOMPRA'])
dfAcoes = dfAcoes.astype( 
    dtype={
        'SETORECONOMICO': str,
        'SUBSETOR': str,
        'SEGMENTO':str,
        'EMPRESA':str,
        'TITULO':str,
        'ACAO':str,
        'QTDECOTAS':int,
        'VALOR':'Float64',
        'DATACOMPRA':str
        }
    )

#Dataframe das ações não localizadas para analisar a razão de não ter lolizado essa ação
dfAcoesNaoCarregadas = pd.DataFrame(columns = ['TITULO','CODIGO', 'COTAS', 'VALOR', 'DATA COMPRA'])

### Carregando informação de Setor e Segmento das ações. 

Esse trecho do código utiliza a biblioteca **tradingcomdados**

In [4]:
for cod in acoes_ibov['CODIGO']:

    titulo      = str(acoes_ibov['TITULO CORRETORA'][acoes_ibov['CODIGO'] == cod].values[0])
    acao        = str(acoes_ibov['CODIGO'][acoes_ibov['CODIGO'] == cod].values[0])
    cotas       = str(acoes_ibov['COTAS'][acoes_ibov['CODIGO'] == cod].values[0])
    valor       = str(acoes_ibov['VALOR'][acoes_ibov['CODIGO'] == cod].values[0])
    dtcompra    = acoes_ibov['DATA'][acoes_ibov['CODIGO'] == cod].values[0]    

    #Retorna os detalhes de Segmento das ações
    dfSect = ad.get_sectors(acao) 
    
    try:
        setor       = dfSect['SETOR ECONÔMICO'].values[0]
        subsetor    = dfSect['SUBSETOR'].values[0]
        segmento    = dfSect['SEGMENTO'].values[0]
        empresa     = dfSect['NOME NO PREGÃO'].values[0]

        dfAcoes.loc[-1] = [setor, subsetor, segmento, empresa, titulo, acao, cotas, valor, dtcompra]
        dfAcoes.index = dfAcoes.index + 1
        dfAcoes = dfAcoes.sort_index()

    except:

        dfAcoesNaoCarregadas.loc[-1] = [titulo, acao, cotas, valor, dtcompra]
        dfAcoesNaoCarregadas.index = dfAcoesNaoCarregadas.index + 1
        dfAcoesNaoCarregadas = dfAcoesNaoCarregadas.sort_index()

In [5]:
#Deduplicação de linhas e ordena pela coluna do nome da Ação.
dfAcoes = dfAcoes.drop_duplicates().sort_values(by=['ACAO']).reset_index()

In [6]:
# *******************************************************************************************************************************************
# Tabela de histórico de Cotações de Ações 

historico_cotacao = pd.DataFrame(columns = ['ACAO','DATACOTACAO','VALORABERTURA','MAIORCOTACAO','MENORCOTACAO','VALORFECHAMENTO','VALORAJUSTADO','DATAPROCESSAMENTO'])
historico_cotacao = historico_cotacao.astype( 
    dtype={
        'ACAO': str,
        'DATACOTACAO': 'datetime64[ns]',
        'VALORABERTURA':'Float64',
        'MAIORCOTACAO':'Float64',
        'MENORCOTACAO':'Float64',
        'VALORFECHAMENTO':'Float64',
        'VALORAJUSTADO':'Float64',
        'DATAPROCESSAMENTO':'datetime64[ns]'
        }
    )

# *******************************************************************************************************************************************
# Tabela de histórico de Dividendos

dividendos = pd.DataFrame(columns = ['ACAO','DATACOTACAO','VALORDIVIDENDO','FLAG_DIVIDENDOS'])
dividendos = dividendos.astype( 
    dtype={
        'ACAO': str,
        'DATACOTACAO': 'datetime64[ns]',
        'VALORDIVIDENDO':'Float64',
        'FLAG_DIVIDENDOS':str
        }
    )

### Carregando histórico de cotações e dividendos das ações

Esse trecho do código utiliza a biblioteca **yahooquery** para retornar o histórico das ações

In [7]:
end_date =  dt.datetime.today().strftime('%Y-%m-%d %H:%M')

for i in dfAcoes.index:

    tkt = dfAcoes['ACAO'].values[i] + '.SA'
    dtcompra  = pd.to_datetime(dfAcoes['DATACOMPRA'].values[i],errors='ignore', format='%Y-%m-%d')

    #Historico das ações 
    hist = Ticker(str(tkt)).history(start=dtcompra, end=end_date).reset_index()
    
    #Historico de dividendos
    hst_dvd = Ticker(str(tkt)).dividend_history(start=dtcompra, end=end_date).reset_index()

    for x in hist.index:

        acao = tkt.replace('.SA','')
        dtcp = pd.to_datetime(hist['date'].values[x],errors='ignore', format='%Y-%m-%d %H:%M')
        vlop = round(hist['open'].values[x],2)
        vlhg = round(hist['high'].values[x],2)
        vllw = round(hist['low'].values[x],2)
        vlcl = round(hist['close'].values[x],2)
        vlad = round(hist['adjclose'].values[x],2)

        historico_cotacao.loc[-1] = [acao, dtcp, vlop, vlhg, vllw, vlcl, vlad, end_date]
        historico_cotacao.index = historico_cotacao.index + 1
        historico_cotacao = historico_cotacao.sort_index()

    try: 
        # Retorna as datas e valores de dividendos
        for y in hst_dvd.index: 
            dtdiv = pd.to_datetime(hst_dvd['date'].values[y],errors='ignore', format='%Y-%m-%d %H:%M')
            vldiv = round(hst_dvd['dividends'].values[y],2)   

            dividendos.loc[-1] = [acao, dtdiv, vldiv, 'Sim']
            dividendos.index = dividendos.index + 1
            dividendos = dividendos.sort_index()                    

    except:
        pass


In [19]:
# Consolidando Ações Detalhadas com as suas respectivas cotações
CotacaoConsolidadas = pd.merge(dfAcoes,historico_cotacao,on='ACAO')
CotacaoConsolidadas = CotacaoConsolidadas.drop("index", axis='columns')

In [20]:
# Consolidando a tabela de Cotação com os valores de Dividendos
CotacaoConsolidadas = pd.merge(CotacaoConsolidadas, dividendos, on=['ACAO','DATACOTACAO'], how='left') 

# Tratamento de valores de dividendos nulos
CotacaoConsolidadas['VALORDIVIDENDO'] = CotacaoConsolidadas['VALORDIVIDENDO'].fillna(value=0)
CotacaoConsolidadas['FLAG_DIVIDENDOS'] = CotacaoConsolidadas['FLAG_DIVIDENDOS'].fillna(value='Não')

#### **Extraindo o resultado para planilha Excel (.xlsx)**

**Layout:** 
* **SETORECONOMICO**    = Descrição do Titulo na corretora
* **SUBSETOR**          = Código da ação para negociação na Bolsa
* **SEGMENTO**          = Quantidade de fração/Cotas compradas
* **TITULO**            = Valor pago na Cota/Fração da Ação
* **ACAO**              = Data da compra da Cota/Fração 
* **QTDECOTAS**         = Data da compra da Cota/Fração
* **VALOR**             = Data da compra da Cota/Fração
* **DATACOMPRA**        = Data da compra da Cota/Fração
* **DATACOTACAO**       = Data da compra da Cota/Fração
* **VALORABERTURA**     = Data da compra da Cota/Fração
* **MAIORCOTACAO**      = Data da compra da Cota/Fração
* **MENORCOTACAO**      = Data da compra da Cota/Fração
* **VALORAJUSTADO**     = Data da compra da Cota/Fração
* **DATAPROCESSAMENTO** = Data da compra da Cota/Fração
* **VALORDIVIDENDO**    = Data da compra da Cota/Fração
* **FLAG_DIVIDENDOS**   = Data da compra da Cota/Fração


In [23]:
outputname = 'Outputs/MinhasCotacoes.xlsx'

# Usando o ExcelWriter, cria um doc .xlsx, usando engine='xlsxwriter'
MinhasCotacoes = pd.ExcelWriter(outputname, engine='xlsxwriter')

# Armazena cada df em uma planilha diferente do mesmo arquivo
CotacaoConsolidadas.to_excel(MinhasCotacoes, sheet_name='Cotação das Ações', index=False)
dfAcoesNaoCarregadas.to_excel(MinhasCotacoes, sheet_name='Titulos não Localizados', index=False)

# Fecha o ExcelWriter e gera o arquivo .xlsx
MinhasCotacoes._save()

In [24]:
CotacaoConsolidadas

,SETORECONOMICO,SUBSETOR,SEGMENTO,EMPRESA,TITULO,ACAO,QTDECOTAS,VALOR,DATACOMPRA,DATACOTACAO,VALORABERTURA,MAIORCOTACAO,MENORCOTACAO,VALORFECHAMENTO,VALORAJUSTADO,DATAPROCESSAMENTO,VALORDIVIDENDO,FLAG_DIVIDENDOS
0,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-24,11.96,11.99,11.87,11.87,11.87,2024-05-26 22:37,0.0,Não
1,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-23,11.92,12.07,11.92,11.96,11.96,2024-05-26 22:37,0.0,Não
2,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-22,11.98,12.11,11.93,11.97,11.97,2024-05-26 22:37,0.0,Não
3,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-21,12.10,12.25,12.02,12.02,12.02,2024-05-26 22:37,0.0,Não
4,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,AMBEV S/A,AMBEVS/A ON,ABEV3,10,17.65,2019-06-06,2024-05-20,12.33,12.39,12.05,12.10,12.10,2024-05-26 22:37,0.0,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32523,Materiais Básicos,Mineração,Minerais Metálicos,VALE,VALE ONNM,VALE3,29,54.72,2020-02-07,2020-02-13,52.22,52.40,51.56,52.14,34.95,2024-05-26 22:37,0.0,Não
32524,Materiais Básicos,Mineração,Minerais Metálicos,VALE,VALE ONNM,VALE3,29,54.72,2020-02-07,2020-02-12,53.10,53.38,52.83,53.07,35.57,2024-05-26 22:37,0.0,Não
32525,Materiais Básicos,Mineração,Minerais Metálicos,VALE,VALE ONNM,VALE3,29,54.72,2020-02-07,2020-02-11,51.15,52.20,51.02,52.05,34.89,2024-05-26 22:37,0.0,Não
32526,Materiais Básicos,Mineração,Minerais Metálicos,VALE,VALE ONNM,VALE3,29,54.72,2020-02-07,2020-02-10,51.90,51.90,49.92,50.19,33.64,2024-05-26 22:37,0.0,Não


In [13]:
# ## Excluindo os Dataframes para limpar a memoria
# del acoes_ibov
# del dfAcoesNaoCarregadas
# del historico_cotacao
# del CotacaoConsolidadas
# del MinhasCotacoes
del dividendos